# LLM Iteration 2

**Aim**

Print out patient type when running the model output.

**Prompt**

Update this model to print out the patient type when running the model.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np

In [2]:
class StrokeWard:
    def __init__(self, env, num_beds):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        ##################################

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            ##################################

            print(f'Patient {patient.id} ({patient.patient_type}) admitted to '
                  f'the stroke ward at day {self.env.now}')
            # Length of stay for all patients is 7 days
            yield self.env.timeout(7)
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            ##################################
            print(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {self.env.now}')

class Patient:
    def __init__(self, id, patient_type):
        self.id = id
        self.patient_type = patient_type

def stroke_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(1.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Stroke")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def tia_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(9.3)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "TIA")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def complex_neurological_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(3.6)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Complex Neurological")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def other_arrival(env, stroke_ward):
    while True:
        interarrival_time = np.random.default_rng().exponential(3.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Other")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

In [3]:
env = simpy.Environment()
stroke_ward = StrokeWard(env, num_beds=10)

env.process(stroke_arrival(env, stroke_ward))
env.process(tia_arrival(env, stroke_ward))
env.process(complex_neurological_arrival(env, stroke_ward))
env.process(other_arrival(env, stroke_ward))

# Set the run length to 2 months (default)
run_length_days = 60 * 2

print("Simulation started...")
env.run(until=run_length_days)
print("Simulation ended.")

Simulation started...
Patient 0 (Stroke) admitted to the stroke ward at day 0.9006536937820486
Patient 1 (Stroke) admitted to the stroke ward at day 3.8576466626553785
Patient 2 (Stroke) admitted to the stroke ward at day 4.127969511920219
Patient 3 (Complex Neurological) admitted to the stroke ward at day 4.32981572288223
Patient 4 (Stroke) admitted to the stroke ward at day 4.4406722697844545
Patient 5 (Other) admitted to the stroke ward at day 4.818170808581365
Patient 6 (Stroke) admitted to the stroke ward at day 4.9854440722860485
Patient 7 (Complex Neurological) admitted to the stroke ward at day 5.008792527835208
Patient 8 (Stroke) admitted to the stroke ward at day 5.547166303031286
Patient 9 (Complex Neurological) admitted to the stroke ward at day 6.168569476693557
Patient 0 (Stroke) discharged from the stroke ward at day 7.900653693782049
Patient 10 (TIA) admitted to the stroke ward at day 7.900653693782049
Patient 1 (Stroke) discharged from the stroke ward at day 10.8576466

This updated Python code includes printing out the patient type when running the simulation model of a stroke ward in a hospital using SimPy 4. The model now displays the patient type along with the patient ID during admission and discharge events.

**Checks**

In [4]:
class TestCase:
    def __init__(self, env, ward):
        self.env = env
        self.ward = ward
        self.env.process(stroke_arrival(env, ward))
        self.env.process(tia_arrival(env, ward))
        self.env.process(complex_neurological_arrival(env, ward))
        self.env.process(other_arrival(env, ward))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)

        for los in ward.los_store:
            if round(los, 10) > 7:
                print('Patient length of stay exceeded limit.')
            elif round(los, 10) < 7:
                print('Patient did not stay for the full duration.')

In [5]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward)
test.test_run_length()

Patient 0 (Other) admitted to the stroke ward at day 0.3118697484981019
Patient 1 (Stroke) admitted to the stroke ward at day 1.3702174658388322
Patient 2 (Stroke) admitted to the stroke ward at day 3.3685634772098716
Patient 3 (Complex Neurological) admitted to the stroke ward at day 3.413538934092374
Patient 4 (Stroke) admitted to the stroke ward at day 4.379691592004491
Patient 5 (Other) admitted to the stroke ward at day 5.5887693184904315
Patient 6 (Other) admitted to the stroke ward at day 5.892912357337556
Patient 7 (Stroke) admitted to the stroke ward at day 6.584765428830515
Patient 0 (Other) discharged from the stroke ward at day 7.311869748498102
Patient 8 (Complex Neurological) admitted to the stroke ward at day 7.84740797981447
Patient 9 (Stroke) admitted to the stroke ward at day 7.891584693371344
Patient 1 (Stroke) discharged from the stroke ward at day 8.370217465838833
Patient 10 (TIA) admitted to the stroke ward at day 8.44651079879941
Patient 11 (Stroke) admitted to 

In [6]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward)
test.test_bed_hold()

Patient 0 (Other) admitted to the stroke ward at day 0.14174834029529734
Patient 0 (Other) discharged from the stroke ward at day 7.1417483402952975
Patient 1 (Stroke) admitted to the stroke ward at day 7.1417483402952975
Patient 1 (Stroke) discharged from the stroke ward at day 14.141748340295297
Patient 2 (Stroke) admitted to the stroke ward at day 14.141748340295297
Patient 2 (Stroke) discharged from the stroke ward at day 21.141748340295297
Patient 3 (Stroke) admitted to the stroke ward at day 21.141748340295297
Patient 3 (Stroke) discharged from the stroke ward at day 28.141748340295297
Patient 4 (Complex Neurological) admitted to the stroke ward at day 28.141748340295297
Patient 4 (Complex Neurological) discharged from the stroke ward at day 35.141748340295294
Patient 5 (Complex Neurological) admitted to the stroke ward at day 35.141748340295294
Patient 5 (Complex Neurological) discharged from the stroke ward at day 42.141748340295294
Patient 6 (TIA) admitted to the stroke ward a

In [7]:
# Checks if stay length correctly implemented
NUM_BEDS = 10
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward)
test.test_length_of_stay()

Patient 0 (Other) admitted to the stroke ward at day 0.38802559698636324
Patient 1 (TIA) admitted to the stroke ward at day 1.3708065065165624
Patient 2 (Stroke) admitted to the stroke ward at day 1.9453015012158348
Patient 3 (Stroke) admitted to the stroke ward at day 1.9592258438623587
Patient 4 (Complex Neurological) admitted to the stroke ward at day 2.647149246104384
Patient 5 (Complex Neurological) admitted to the stroke ward at day 3.3903882238154313
Patient 6 (TIA) admitted to the stroke ward at day 4.544967013669186
Patient 7 (TIA) admitted to the stroke ward at day 6.4873336845505065
Patient 8 (Complex Neurological) admitted to the stroke ward at day 7.126916689151193
Patient 9 (Stroke) admitted to the stroke ward at day 7.223610993526416
Patient 0 (Other) discharged from the stroke ward at day 7.388025596986363
Patient 10 (Other) admitted to the stroke ward at day 7.641719016525617
Patient 1 (TIA) discharged from the stroke ward at day 8.370806506516562
Patient 11 (Stroke) a

**Reflection**

The LLM was able to edit the code as instructed to print out the patient type, which ensures that all patient types are arriving given their different interarrival times. All the checks still work, so this has not changed how the code is implemented.